# Data Exploration

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 30/10/2025   | Martin | Create  | Creating an ML dashboard for housing price prediction | 

# Content

* [Load Data](#load-data)
* [Split Data](#data-split)

# Load Data

In [16]:
import pandas as pd

In [17]:
df = pd.read_csv("../data/raw/HouseTS.csv")
metros = pd.read_csv("../data/raw/usmetros.csv")

In [18]:
df.head()

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,...,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
1,2012-04-30,61870.0,245000.0,40.723982,130.528256,22.0,29.0,56.0,69.0,89.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202421.064584,Atlanta-Sandy Springs-Alpharetta
2,2012-05-31,125500.0,217450.0,63.913043,119.919216,24.0,40.0,63.0,60.0,144.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202681.309539,Atlanta-Sandy Springs-Alpharetta
3,2012-06-30,153000.0,189900.0,81.598080,105.617353,34.0,46.0,50.0,57.0,126.0,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202998.603897,Atlanta-Sandy Springs-Alpharetta
4,2012-07-31,165500.0,154000.0,81.598080,83.921175,39.0,49.0,42.0,50.0,80.0,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,203781.903446,Atlanta-Sandy Springs-Alpharetta


In [19]:
df['date'] = pd.to_datetime(df['date'])
print(df['date'].dt.year.value_counts())

date
2013    74712
2014    74712
2015    74712
2016    74712
2017    74712
2018    74712
2019    74712
2020    74712
2021    74712
2022    74712
2023    74712
2012    62260
Name: count, dtype: int64


# Data Split

- Training data (2012-2019)
- Validation data (2020-2021)
- Testing data (2022-2023)

In [20]:
# Sort by date
df = df.sort_values("date")

# Define cutoff dates
cutoff_date_train = "2019-12-31"
cutoff_date_validation = "2021-12-31"

# Train data
train_df = df[df['date'] <= cutoff_date_train]
valid_df = df[(df['date'] > cutoff_date_train) & (df['date'] <= cutoff_date_validation)]
test_df = df[df['date'] > cutoff_date_validation]

print(f"Training data shape: {train_df.shape}")
print(f"Validation data shape: {valid_df.shape}")
print(f"Testing data shape: {test_df.shape}")

Training data shape: (585244, 39)
Validation data shape: (149424, 39)
Testing data shape: (149424, 39)


In [21]:
# train_df.to_csv("../data/raw/train.csv", index=False)
# test_df.to_csv("../data/raw/test.csv", index=False)
# valid_df.to_csv("../data/raw/valid.csv", index=False)

# Data Exploration & Cleaning

Replacing `city_full` with latitude and longitude information, since there are too many to count

In [22]:
train_df['city_full'].value_counts()

city_full
New York-Newark-Jersey City            78020
Chicago-Naperville-Elgin               35344
Los Angeles-Long Beach-Anaheim         33840
Philadelphia-Camden-Wilmington         31396
DC_Metro                               29516
Pittsburgh                             27824
Boston-Cambridge-Newton                25568
Dallas-Fort Worth-Arlington            23594
Houston-The Woodlands-Sugar Land       20586
Minneapolis-St. Paul-Bloomington       20398
Detroit-Warren-Dearborn                20022
St. Louis                              19834
Atlanta-Sandy Springs-Alpharetta       19082
Miami-Fort Lauderdale-Pompano Beach    17014
San Francisco-Oakland-Berkeley         15604
Seattle-Tacoma-Bellevue                14664
Phoenix-Mesa-Chandler                  14006
Cincinnati                             14006
Baltimore-Columbia-Towson              13818
Riverside-San Bernardino-Ontario       13724
Tampa-St. Petersburg-Clearwater        12126
Denver-Aurora-Lakewood                 11750


In [23]:
# Remap some city names
city_mapping = {
  'Las Vegas-Henderson-Paradise': 'Las Vegas-Henderson-North Las Vegas',
  'Denver-Aurora-Lakewood': 'Denver-Aurora-Centennial',
  'Houston-The Woodlands-Sugar Land': 'Houston-Pasadena-The Woodlands',
  'Austin-Round Rock-Georgetown': 'Austin-Round Rock-San Marcos',
  'Miami-Fort Lauderdale-Pompano Beach': 'Miami-Fort Lauderdale-West Palm Beach',
  'San Francisco-Oakland-Berkeley': 'San Francisco-Oakland-Fremont',
  'DC_Metro': 'Washington-Arlington-Alexandria',
  'Atlanta-Sandy Springs-Alpharetta': 'Atlanta-Sandy Springs-Roswell'
}

def clean_and_merge(df: pd.DataFrame) -> pd.DataFrame:
  """Apply city name fixes, merge lat/lng from metros, drop dup col."""
  df["city_full"] = df["city_full"].replace(city_mapping)
  
  df = df.merge(
    metros[["metro_full", "lat", "lng"]],
    how="left",
    left_on="city_full",
    right_on="metro_full"
  )
  df.drop(columns=["metro_full"], inplace=True)

  # Log any cities that still didn’t match
  missing = df[df["lat"].isnull()]["city_full"].unique()
  if len(missing) > 0:
    print("⚠️ Still missing lat/lng for:", missing)
  else:
    print("✅ All cities matched with metros dataset.")

  return df

In [24]:
train_df = clean_and_merge(train_df)
valid_df = clean_and_merge(valid_df)

✅ All cities matched with metros dataset.
✅ All cities matched with metros dataset.


C:\Users\User\AppData\Local\Temp\ipykernel_20240\636688607.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["city_full"] = df["city_full"].replace(city_mapping)
C:\Users\User\AppData\Local\Temp\ipykernel_20240\636688607.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["city_full"] = df["city_full"].replace(city_mapping)


Check and remove duplicates

- Some houses have their information shown on a later date, thus have multiple entries where the information is exactly the same less the date and year. So these are removed

In [30]:
print(train_df.shape)

duplicated_rows = train_df[train_df.duplicated()].shape[0]
print("duplicated_rows:", duplicated_rows)

duplicated_rows = train_df[train_df.duplicated(subset=train_df.columns.difference(['date', 'year']))].shape[0]
print("duplicated_rows excluding date column:", duplicated_rows)

(585244, 41)
duplicated_rows: 0
duplicated_rows excluding date column: 6321


In [31]:
# Delete duplicates
train_df = train_df.drop_duplicates(subset=train_df.columns.difference(['date', 'year']), keep='last')

print(train_df.shape)

duplicated_rows = train_df[train_df.duplicated()].shape[0]
print("duplicated_rows:", duplicated_rows)

duplicated_rows = train_df[train_df.duplicated(subset=train_df.columns.difference(['date', 'year']))].shape[0]
print("duplicated_rows excluding date column:", duplicated_rows)

(578923, 41)
duplicated_rows: 0
duplicated_rows excluding date column: 0


41

In [2]:
%watermark

Last updated: 2025-06-18T19:03:45.452311+08:00

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.31.0

Compiler    : MSC v.1938 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 183 Stepping 1, GenuineIntel
CPU cores   : 20
Architecture: 64bit

